# Introduction

# Libraries

In [5]:
import numpy as np
import pandas as pd
import os 
import matplotlib.pyplot as plt

from PIL import Image
from sklearn.datasets import fetch_lfw_people
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout

# Data loading

In [2]:
images = fetch_lfw_people()